In [1]:
%load_ext tensorboard

In [2]:
import tensorflow as tf
import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import datetime

from tqdm import tqdm

2024-07-28 20:52:03.521230: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-28 20:52:03.548871: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-28 20:52:03.548909: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-28 20:52:03.549700: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-28 20:52:03.556150: I tensorflow/core/platform/cpu_feature_guar

In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Model

from sklearn.preprocessing import OneHotEncoder

In [4]:
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'
print(os.getenv('TF_GPU_ALLOCATOR'))

cuda_malloc_async


In [5]:
class VGGNet(Sequential):
    def __init__(self, input_shape, num_classes):
        super().__init__()
        
        self.add(Conv2D(
            64,
            kernel_size=(3,3),
            kernel_initializer="he_normal",
            padding="same",
            activation="relu",
            input_shape=input_shape
        ))

        self.add(Conv2D(
            64,
            kernel_size=(3,3),
            kernel_initializer="he_normal",
            padding="same",
            activation="relu"
        ))

        self.add(MaxPooling2D(
            pool_size=(2, 2),
            strides=2
        ))

        self.add(Conv2D(
            128,
            kernel_size=(3,3),
            kernel_initializer="he_normal",
            padding="same",
            activation="relu"
        ))

        self.add(Conv2D(
            128,
            kernel_size=(3,3),
            kernel_initializer="he_normal",
            padding="same",
            activation="relu"
        ))

        self.add(MaxPooling2D(
            pool_size=(2, 2),
            strides=2
        ))

        self.add(Conv2D(
            256,
            kernel_size=(3,3),
            kernel_initializer="he_normal",
            padding="same",
            activation="relu"
        ))

        self.add(Conv2D(
            256,
            kernel_size=(3,3),
            kernel_initializer="he_normal",
            padding="same",
            activation="relu"
        ))

        self.add(Conv2D(
            256,
            kernel_size=(3,3),
            kernel_initializer="he_normal",
            padding="same",
            activation="relu"
        ))

        self.add(MaxPooling2D(
            pool_size=(2, 2),
            strides=2
        ))

        self.add(Conv2D(
            512,
            kernel_size=(3,3),
            kernel_initializer="he_normal",
            padding="same",
            activation="relu"
        ))

        self.add(Conv2D(
            512,
            kernel_size=(3,3),
            kernel_initializer="he_normal",
            padding="same",
            activation="relu"
        ))

        self.add(Conv2D(
            512,
            kernel_size=(3,3),
            kernel_initializer="he_normal",
            padding="same",
            activation="relu"
        ))

        self.add(MaxPooling2D(
            pool_size=(2, 2),
            strides=2
        ))

        self.add(Conv2D(
            512,
            kernel_size=(3,3),
            kernel_initializer="he_normal",
            padding="same",
            activation="relu"
        ))

        self.add(Conv2D(
            512,
            kernel_size=(3,3),
            kernel_initializer="he_normal",
            padding="same",
            activation="relu"
        ))

        self.add(Conv2D(
            512,
            kernel_size=(3,3),
            kernel_initializer="he_normal",
            padding="same",
            activation="relu"
        ))

        self.add(MaxPooling2D(
            pool_size=(2, 2),
            strides=2
        ))

        self.add(Flatten()),

        self.add(Dense(4096, activation="relu"))
        self.add(Dense(4096, activation="relu"))
        self.add(Dense(1000, activation="softmax"))

        self.compile(
            optimizer=Adam(0.001),
            loss="categorical_crossentropy",
            metrics=["accuracy"]
        )

In [6]:
num_epochs = 1
batch_size = 2
img_height = 224
img_width = 224
Input_shape = (img_width, img_height, 3)
num_classes = 3


model = VGGNet(Input_shape, num_classes=num_classes)

model.summary()

2024-07-28 20:52:06.196972: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:274] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2024-07-28 20:52:06.196993: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:129] retrieving CUDA diagnostic information for host: abhinav-Nitro-AN515-58
2024-07-28 20:52:06.196996: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:136] hostname: abhinav-Nitro-AN515-58
2024-07-28 20:52:06.197102: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:159] libcuda reported version is: 535.183.1
2024-07-28 20:52:06.197111: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:163] kernel reported version is: 535.183.1
2024-07-28 20:52:06.197113: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:241] kernel version seems to match DSO: 535.183.1


Model: "vgg_net"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 64)      1792      
                                                                 
 conv2d_1 (Conv2D)           (None, 224, 224, 64)      36928     
                                                                 
 max_pooling2d (MaxPooling2  (None, 112, 112, 64)      0         
 D)                                                              
                                                                 
 conv2d_2 (Conv2D)           (None, 112, 112, 128)     73856     
                                                                 
 conv2d_3 (Conv2D)           (None, 112, 112, 128)     147584    
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 56, 56, 128)       0         
 g2D)                                                      

In [7]:
train_images_dir = "/home/abhinav/Documents/Work/2 Hobby_projects/Datasets/Imagenet/synsets"
valid_images_dir = "/home/abhinav/Documents/Work/2 Hobby_projects/Datasets/Imagenet/valid_synsets"


train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1
)

train_generator = train_datagen.flow_from_directory(
    train_images_dir,
    target_size=(img_width, img_height),
    color_mode="rgb",
    batch_size=batch_size,
    shuffle=True,
    seed=1,
    class_mode="categorical"
)

valid_datagen = ImageDataGenerator(
    rescale=1./255
)

valid_generator = valid_datagen.flow_from_directory(
    valid_images_dir,
    target_size=(img_width, img_height),
    color_mode="rgb",
    batch_size=batch_size,
    shuffle=True,
    seed=1,
    class_mode="categorical"
)

train_images_num = train_generator.samples
valid_images_num = valid_generator.samples

Found 3605 images belonging to 3 classes.
Found 264 images belonging to 3 classes.


In [8]:
log_dir = "./logs/fit" + datetime.datetime.now().strftime("%d%M%Y-%H%M%S")
tensorboard_callbacks = tf.keras.callbacks.TensorBoard(log_dir=log_dir)
callbacks_list = [tensorboard_callbacks]

In [9]:
# Since lack of gpu power, we cant train this model in local machine, so we take step to take pretrained model

# model.fit(
#     train_generator,
#     epochs=num_epochs,
#     steps_per_epoch=train_images_num//batch_size,
#     validation_data=valid_generator,
#     validation_steps=valid_images_num//batch_size,
#     callbacks=callbacks_list,
#     verbose=1
# )

In [10]:
from keras.applications.vgg16 import VGG16
vggmodel = VGG16(weights='imagenet', include_top=True)

553467096/553467096 [==============================] - 51s 0us/step


In [16]:
print("layers of the model:")
for la in vggmodel.layers:
    print(la)


print("\n\nlayers that we are going to set trainable to False")
for layers in (vggmodel.layers)[:15]:
    print(layers)
    layers.trainable = False

X= vggmodel.layers[-2].output
predictions = Dense(2, activation="softmax")(X)
model_final = Model(vggmodel.input, predictions)
opt = Adam(learning_rate=0.0001)
model_final.compile(loss = keras.losses.categorical_crossentropy, optimizer = opt, metrics=["accuracy"])
model_final.summary()

layers of the model:


layers that we are going to set trainable to False
Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                   

                                                                 
 block3_conv2 (Conv2D)       (None, 56, 56, 256)       590080    
                                                                 
 block3_conv3 (Conv2D)       (None, 56, 56, 256)       590080    
                                                                 
 block3_pool (MaxPooling2D)  (None, 28, 28, 256)       0         
                                                                 
 block4_conv1 (Conv2D)       (None, 28, 28, 512)       1180160   
                                                                 
 block4_conv2 (Conv2D)       (None, 28, 28, 512)       2359808   
                                                                 
 block4_conv3 (Conv2D)       (None, 28, 28, 512)       2359808   
                                                                 
 block4_pool (MaxPooling2D)  (None, 14, 14, 512)       0         
                                                                 
 block5_co